In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3336/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_3336/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_3_6_4,0.726515,0.241298,-0.096404,0.006082,0.441445,5.073447,3.921774,1.009056,1.101095,0.664413,-0.166868,0.674877,99.635404,206.180380,"Hidden Size=[12], regularizer=0.02, learning_r..."
1,model_3_6_3,0.723387,0.230089,-0.199629,0.024036,0.446495,5.148396,4.291004,0.990828,1.134155,0.668202,-0.180216,0.678726,99.612656,206.157633,"Hidden Size=[12], regularizer=0.02, learning_r..."
2,model_3_6_0,0.694038,0.222326,-0.938591,0.130885,0.493868,5.200309,6.934227,0.882352,1.283125,0.702757,-0.305438,0.713825,99.410974,205.955950,"Hidden Size=[12], regularizer=0.02, learning_r..."
3,model_3_37_2,0.763916,0.210197,0.133335,0.387761,0.381074,5.281419,3.100010,0.460747,1.251570,0.617312,-0.007290,0.627034,99.929524,206.474500,"Hidden Size=[12], regularizer=0.02, learning_r..."
4,model_3_36_7,0.754606,0.209710,0.156848,-0.873563,0.396102,5.284673,3.015907,0.509114,1.251246,0.629367,-0.047014,0.639278,99.852166,206.397142,"Hidden Size=[12], regularizer=0.02, learning_r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,model_3_23_2,0.727216,-0.827175,-0.301564,-6.477445,0.440314,12.218324,4.655619,0.493068,1.224641,0.663561,-0.163878,0.674012,99.640535,206.185511,"Hidden Size=[12], regularizer=0.02, learning_r..."
727,model_3_23_3,0.728817,-0.860163,-0.300821,-6.510449,0.437729,12.438918,4.652961,0.495244,1.222580,0.661611,-0.157046,0.672031,99.652311,206.197287,"Hidden Size=[12], regularizer=0.02, learning_r..."
728,model_3_7_4,0.726547,-0.980799,-14.302684,0.107147,0.441394,13.245612,54.736813,0.632733,0.975766,0.664375,-0.166732,0.674838,99.635637,206.180613,"Hidden Size=[12], regularizer=0.02, learning_r..."
729,model_3_7_6,0.722392,-1.359832,-19.624418,0.007111,0.448100,15.780210,73.772346,0.703624,0.970616,0.669403,-0.184460,0.679945,99.605477,206.150453,"Hidden Size=[12], regularizer=0.02, learning_r..."
